In [1]:
#!pip install web3

In [2]:
#!pip install multicall

In [3]:
from web3 import Web3
from multicall import Call, Multicall
        
GearboxAddressProvider = Web3.toChecksumAddress('0xcF64698AFF7E5f27A11dff868AF228653ba53be0')
yVaultUSDC = Web3.toChecksumAddress('0xa354F35829Ae975e850e23e9615b11Da1B3dC4DE')

# The dafault RPC from ethersjs, change it if it doesn't work: https://infura.io/docs
RPC_Endpoint = 'https://mainnet.infura.io/v3/84842078b09946638c03157f83405213'

ABI = """[{"name":"getAccountFactory",
           "inputs":[],
           "outputs":[{"internalType":"address","name":"","type":"address"}],
           "stateMutability":"view","type":"function"},
          {"name":"countCreditAccounts",
           "inputs":[],
           "outputs":[{"internalType":"uint256","name":"","type":"uint256"}],
           "stateMutability":"view","type":"function"},
          {"name":"decimals",
           "inputs":[],
           "outputs":[{"name":"","type":"uint256"}],
           "stateMutability":"view","type":"function"
           }
          ]
      """

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

w3_eth = Web3(Web3.HTTPProvider(RPC_Endpoint, request_kwargs={'timeout': 20}))
print ('Ethereum connected:', w3_eth.isConnected())

Ethereum connected: True


In [4]:
AccountFactory = w3_eth.eth.contract(address=GearboxAddressProvider, abi=ABI).functions.getAccountFactory().call()
print('AccountFactory:', AccountFactory)

countCreditAccounts = w3_eth.eth.contract(address=AccountFactory, abi=ABI).functions.countCreditAccounts().call()
print('countCreditAccounts:', countCreditAccounts)

yVaultUSDC_decimals = w3_eth.eth.contract(address=yVaultUSDC, abi=ABI).functions.decimals().call()
print('yVaultUSDC_decimals:', yVaultUSDC_decimals)


AccountFactory: 0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04
countCreditAccounts: 5001
yVaultUSDC_decimals: 6


In [5]:
idList = list(range(countCreditAccounts))
multi_idCA = {}

for ids in list(chunks(idList, 400)): #chunk size for multicall = 400
    #d_ca = get_data_multicall(df.loc[id_range], 'creditAccounts', df_abi, AccountFactory)
    multi_result = Multicall([
                    Call(AccountFactory, ['creditAccounts(uint256)(address)', x], [[x, Web3.toChecksumAddress]]) for x in ids
                    ]
                    ,_w3 = w3_eth)
    
    multi_result = multi_result()
    multi_idCA.update(multi_result)
    

In [6]:
#display(multi_idCA)

In [7]:
multi_idBalance = {}
for ids in list(chunks(list(multi_idCA), 400)): #chunk size for multicall = 400
    multi_result = Multicall([
                    Call(yVaultUSDC, ['balanceOf(address)(uint256)', multi_idCA[x]], [[x, None]]) for x in ids
                    ]
                    ,_w3 = w3_eth)
    
    multi_result = multi_result()
    multi_idBalance.update(multi_result)


In [8]:
#display(multi_idBalance)

In [9]:
multi_CA = {multi_idCA[x]:multi_idBalance[x] for x in multi_idCA}
#display(multi_CA)

In [10]:
#only with existing balance
{key:value for key, value in multi_CA.items() if value > 0}

{'0xfBA6b8817699cc4B90983887F3B96579f366bd5a': 36823268679,
 '0x9b7249E6D6fdd05295d8E5ef68a1068dec1A6d07': 1,
 '0x7a2f98848C626815d93957c1D2637fCE7cCa9fC0': 39799123570,
 '0x8e996336FEe5CE8F6BC3AfD68e5b40f6Cef8cb87': 3952079976,
 '0xfD96Bc2034061e0E02bbB534B4c99C0FED30160D': 39776257801,
 '0xE5AAe8E98607fE92C04e09AD023bcF9dD91999C5': 39655792262,
 '0x9dE7BE4057872Af76a6C3eae5FF1219cce9adAB9': 40691007865,
 '0x60c230b831eeF255Dad4d5244170E98B8F361C21': 3969974252,
 '0xd7FD97d43f790F87bc62A3B3979Df20DE6DDEB2A': 128970198740,
 '0x16cF8A14a0B20e4150F25D7b8e42b653Ef5bca30': 39332161711,
 '0xAB2d65eA191313AEFCD0C48A6f7D6b20852bAE0C': 40785830856,
 '0xaBBd655b3791175113c1f1146D3B369494A2b815': 39639734244,
 '0x5dBCFABBa5482206FfaE8446ae00ce22CA767e46': 19302346058,
 '0x9554E3C31Aa1e2cd43C79C7Fed4d166B39a1aaf6': 39554918657,
 '0x1F95B6AC9BaCcCAF7121152b704D6b3820A81f32': 16037508788}

In [11]:
#decimal values
{key:value*10**(-yVaultUSDC_decimals) for key, value in multi_CA.items() if value > 0}

{'0xfBA6b8817699cc4B90983887F3B96579f366bd5a': 36823.268679,
 '0x9b7249E6D6fdd05295d8E5ef68a1068dec1A6d07': 1e-06,
 '0x7a2f98848C626815d93957c1D2637fCE7cCa9fC0': 39799.123569999996,
 '0x8e996336FEe5CE8F6BC3AfD68e5b40f6Cef8cb87': 3952.079976,
 '0xfD96Bc2034061e0E02bbB534B4c99C0FED30160D': 39776.257801,
 '0xE5AAe8E98607fE92C04e09AD023bcF9dD91999C5': 39655.792261999995,
 '0x9dE7BE4057872Af76a6C3eae5FF1219cce9adAB9': 40691.007865,
 '0x60c230b831eeF255Dad4d5244170E98B8F361C21': 3969.974252,
 '0xd7FD97d43f790F87bc62A3B3979Df20DE6DDEB2A': 128970.19873999999,
 '0x16cF8A14a0B20e4150F25D7b8e42b653Ef5bca30': 39332.161711,
 '0xAB2d65eA191313AEFCD0C48A6f7D6b20852bAE0C': 40785.830856,
 '0xaBBd655b3791175113c1f1146D3B369494A2b815': 39639.734244,
 '0x5dBCFABBa5482206FfaE8446ae00ce22CA767e46': 19302.346058,
 '0x9554E3C31Aa1e2cd43C79C7Fed4d166B39a1aaf6': 39554.918656999995,
 '0x1F95B6AC9BaCcCAF7121152b704D6b3820A81f32': 16037.508788}